In [137]:
import gzip
import _pickle

from s2clientprotocol import common_pb2 as sc_common
from s2clientprotocol import sc2api_pb2 as sc_pb

def load_compressed(filename):
    with gzip.open(filename, 'rb') as f:
        return _pickle.load(f)

data = load_compressed('/Users/dkorduban/workspace/sc2/my/replays/extracted/fastreload20180915T182653.SC2Replay.gz')
info = sc_pb.ResponseReplayInfo()
info.ParseFromString(data['replay_info'])
print(info)
print(len(_pickle.dumps(info)), len(info.SerializeToString()))

player1 = data['player1']

player2 = data['player2']

# obs = sc_pb.ResponseObservation()

# hh = set()
# for x in player1:
#     obs.ParseFromString(x)
#     for u in obs.observation.raw_data.units:
#         if not u.is_flying:
#             print(u.pos)
#             hh.add(u.pos.z)

# print(sorted(hh))




map_name: "Blueshift LE"
local_map_path: "BlueshiftLE.SC2Map"
player_info {
  player_info {
    player_id: 1
    race_requested: Protoss
    race_actual: Protoss
  }
  player_result {
    player_id: 1
    result: Defeat
  }
  player_apm: 1661
}
player_info {
  player_info {
    player_id: 2
    race_requested: Terran
    race_actual: Terran
  }
  player_result {
    player_id: 2
    result: Victory
  }
  player_apm: 57
}
game_duration_loops: 1712
game_duration_seconds: 76.43390655517578
game_version: "4.6.1.68195"
data_build: 68195
base_build: 67926
data_version: "BEA99B4A8E7B41E62ADC06D194801BAB"

217 134


In [140]:
from sc2.game_data import GameData
from sc2.unit import Unit
from sc2.units import Units


data = player1['data']
# data = sc_pb.ResponseData()
# data.ParseFromString(player1['data'])
# print(data)

print(len(_pickle.dumps(data)), len(data.SerializeToString()))
data = GameData(data)
print(data)

# game_info = player1['game_info']
game_info = sc_pb.ResponseGameInfo()
game_info.ParseFromString(player1['game_info'])
print(len(_pickle.dumps(game_info)), len(game_info.SerializeToString()))
print(game_info)

322603 322523
97506 97422
map_name: "Blueshift LE"
local_map_path: "BlueshiftLE.SC2Map"
player_info {
  player_id: 1
  type: Participant
  race_requested: Protoss
  race_actual: Protoss
}
player_info {
  player_id: 2
  type: Computer
  race_requested: Terran
  difficulty: Medium
}
start_raw {
  map_size {
    x: 176
    y: 184
  }
  pathing_grid {
    bits_per_pixel: 8
    size {
      x: 176
      y: 184
    }
    data: "\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\377\3

In [136]:
(15 * 60 * 22.4 / 80) * (267 / 46)

1462.695652173913

In [44]:
import numpy as np
import scipy

In [45]:
import scipy.sparse

In [71]:

scipy.sparse.coo_matrix?

In [128]:
MAX_SIZE = 168
OBS_SIZE = 30


def transform_coords(x, y, h, w, max_size, obs_size):
#     i = np.zeros(len(x), dtype=np.int16)
#     j = np.zeros(len(x), dtype=np.int16)
    x = x * (obs_size / max_size) + obs_size * (max_size - w) / (2 * max_size)
    y = y * (-obs_size / max_size) + obs_size * (max_size + h) / (2 * max_size)
#     i = np.around(y, out=i)
#     j = np.around(x, out=j)
    i = np.around(y)
    j = np.around(x)
    return i, j
    # TODO: coord = max(0, min(obs_size-1, coord)) elementwise
    
def transform_observation(obs, map_size, game_data):
    h, w = map_size
    units = Units.from_proto(obs.observation.raw_data.units, game_data)
    my_units = units.owned
    enemy_units = units.enemy
    
    planes = []
    for units in [my_units, enemy_units]:
        structures = units.structure
        army = units.not_structure
        v = np.zeros(len(structures), dtype=np.float32)
        x = np.zeros(len(structures), dtype=np.float32)
        y = np.zeros(len(structures), dtype=np.float32)
        for idx, u in enumerate(structures):
            v[idx] = (u.health_max + u.shield_max) * u.build_progress
            x[idx] = u.position.y
            y[idx] = u.position.x
        i, j = transform_coords(x, y, h, w, MAX_SIZE, OBS_SIZE)
        sparse_plane = scipy.sparse.coo_matrix((v, (i, j)), shape=(OBS_SIZE, OBS_SIZE))
        planes.append(sparse_plane.todense())
        v = np.zeros(len(army), dtype=np.float32)
        v2 = np.zeros(len(army), dtype=np.float32)
        x = np.zeros(len(army), dtype=np.float32)
        y = np.zeros(len(army), dtype=np.float32)
        for idx, u in enumerate(army):
            v[idx] = u.health + u.shield
            v2[idx] = u._type_data._proto.food_required
            x[idx] = u.position.y
            y[idx] = u.position.x
        i, j = transform_coords(x, y, h, w, MAX_SIZE, OBS_SIZE)
        sparse_plane = scipy.sparse.coo_matrix((v, (i, j)), shape=(OBS_SIZE, OBS_SIZE))
#         print(sparse_plane.todense())
        planes.append(sparse_plane.todense())
        sparse_plane = scipy.sparse.coo_matrix((v2, (i, j)), shape=(OBS_SIZE, OBS_SIZE))
#         print(sparse_plane.todense())
        planes.append(sparse_plane.todense())
    
    result = np.array(planes)
#     print(result.shape)    
    return result

In [135]:
%%time

from itertools import chain

# player2 = data['player2']
s = 0
obs = sc_pb.ResponseObservation()
rollout = []
for x in chain(player1['observations'], player2['observations']):
    obs.ParseFromString(x)
    rollout.append(transform_observation(obs, (160, 160), data))
#     units = Units.from_proto(obs.observation.raw_data.units, data)
#     s += len(units)
#     print(units)
#     for u in obs.observation.raw_data.units:
#         uu = Unit(u, data)
#         print(uu)
# print(len(player2['observations']), s)
rollout = np.array(rollout)
print(rollout.shape)
np.savez_compressed('/Users/dkorduban/workspace/sc2/my/replays/npz/a.npz', both=rollout)
print(10)

(46, 6, 30, 30)
10
CPU times: user 196 ms, sys: 5.94 ms, total: 202 ms
Wall time: 267 ms


In [105]:
np.array.__repr__??